In [ ]:
import pandas as pd
import gspread

from oauth2client.service_account import ServiceAccountCredentials
from utils import*

In [ ]:
LOAD_PRE_PROCESSED_DATA = False

In [ ]:
CREDENTIALS_JSON = 'private-keys/data-6081087c914f.json'
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_JSON, scope)
client = gspread.authorize(creds)

In [ ]:
gc = client.open_by_url('https://docs.google.com/spreadsheets/d/1XH_Si8ZSuKdWNVsBq6vS0L4CaOZ07oLj-mOXDFHeFsw/edit?usp=sharing')

In [ ]:
gc.worksheets()

In [ ]:
routes = gc.get_worksheet_by_id(0)

In [38]:
locations = gc.get_worksheet_by_id(1096112689)

In [39]:
routes_df = pd.DataFrame(routes.get_all_records())


In [40]:
# export routes_df to csv
if not LOAD_PRE_PROCESSED_DATA:
    routes_df.to_csv('export/routes.csv', index=False)

In [41]:
locations_df = pd.DataFrame(locations.get_all_records())

In [42]:
routes_df

,Id,Origin,Destination,Transits 1
0,1,"African Leadership College, Mauritius",Mahogany Shopping Promenade,
1,2,"African Leadership College, Mauritius",Port Louis,Mahogany Shopping Promenade
2,3,"African Leadership College, Mauritius",Pamplemousses,Mahogany Shopping Promenade
3,4,"African Leadership College, Mauritius",Grand baie,Mahogany Shopping Promenade
4,5,"African Leadership College, Mauritius",Flic en flac,Port Louis
5,6,"African Leadership College, Mauritius",Beau bassin,Port Louis
6,7,"African Leadership College, Mauritius","Village Moka, Mauritius",Mahogany Shopping Promenade
7,8,"African Leadership College, Mauritius","SSRN Hospital, Pamplemousses",
8,9,"African Leadership College, Mauritius",Beau Bassin Rose Hill,Port Louis
9,10,"African Leadership College, Mauritius",Quatre Bornes,Beau Bassin Rose Hill


In [43]:
locations_df

,Locations
0,"African Leadership College, Mauritius"
1,Mahogany Shopping Promenade
2,Port Louis
3,Pamplemousses
4,Grand baie
5,Flic en flac
6,Beau bassin
7,"Village Moka, Mauritius"
8,"SSRN Hospital, Pamplemousses"
9,Grand Gaube


In [44]:
# Geocode all locations
if LOAD_PRE_PROCESSED_DATA:
    locations_df_geolocation = pd.read_csv('export/locations_df_geolocation.csv')
else:
    locations_df_geolocation = locations_df.apply(lambda row: parse_geocode_response(row['Locations']), axis='columns', result_type='expand')
    locations_df_geolocation.to_csv('export/locations_df_geolocation.csv')

In [45]:
# Concatenate back the location df and the location with geolocation data pulled from google maps api
locations_df_geolocation_full = pd.concat([locations_df, locations_df_geolocation], axis='columns')

In [46]:
locations_df_geolocation

,longitude,latitude,formatted_address,full_details,success_geocode,match_found_geocode
0,57.562450,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
1,57.576763,-20.099637,"Beau Plan Pamplemousses MU, Pamplemousses 2100...",[{'address_components': [{'long_name': 'Maurit...,True,1
2,57.501222,-20.160891,"Port Louis, Mauritius",[{'address_components': [{'long_name': 'Port L...,True,1
3,57.575926,-20.113601,"Pamplemousses District, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
4,57.581635,-20.008920,"Grand Baie, Mauritius",[{'address_components': [{'long_name': 'Grand ...,True,1
5,57.363424,-20.299535,"Flic En Flac Beach, Mauritius",[{'address_components': [{'long_name': 'Flic E...,True,2
6,57.468383,-20.223031,"Beau Bassin-Rose Hill, Mauritius",[{'address_components': [{'long_name': 'Beau B...,True,1
7,57.502332,-20.219524,"Moka, Mauritius","[{'address_components': [{'long_name': 'Moka',...",True,1
8,57.559292,-20.096699,"B18, Mauritius","[{'address_components': [{'long_name': 'B18', ...",True,1
9,57.669408,-20.014212,"Grand Gaube, Mauritius",[{'address_components': [{'long_name': 'Grand ...,True,1


In [47]:
# dump the result to a csv
locations_df_geolocation_full.to_csv('export/locations_df_geolocation_full.csv')

In [48]:
#gmaps.directions("African Leadership College, Mauritius", "Port Louis")

# Get routes

In [49]:
routes_df

,Id,Origin,Destination,Transits 1
0,1,"African Leadership College, Mauritius",Mahogany Shopping Promenade,
1,2,"African Leadership College, Mauritius",Port Louis,Mahogany Shopping Promenade
2,3,"African Leadership College, Mauritius",Pamplemousses,Mahogany Shopping Promenade
3,4,"African Leadership College, Mauritius",Grand baie,Mahogany Shopping Promenade
4,5,"African Leadership College, Mauritius",Flic en flac,Port Louis
5,6,"African Leadership College, Mauritius",Beau bassin,Port Louis
6,7,"African Leadership College, Mauritius","Village Moka, Mauritius",Mahogany Shopping Promenade
7,8,"African Leadership College, Mauritius","SSRN Hospital, Pamplemousses",
8,9,"African Leadership College, Mauritius",Beau Bassin Rose Hill,Port Louis
9,10,"African Leadership College, Mauritius",Quatre Bornes,Beau Bassin Rose Hill


In [50]:
routes_df_full = routes_df.merge(locations_df_geolocation_full, left_on='Origin', right_on='Locations')

In [51]:
routes_df_full

,Id,Origin,Destination,Transits 1,Locations,longitude,latitude,formatted_address,full_details,success_geocode,match_found_geocode
0,1,"African Leadership College, Mauritius",Mahogany Shopping Promenade,,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
1,2,"African Leadership College, Mauritius",Port Louis,Mahogany Shopping Promenade,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
2,3,"African Leadership College, Mauritius",Pamplemousses,Mahogany Shopping Promenade,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
3,4,"African Leadership College, Mauritius",Grand baie,Mahogany Shopping Promenade,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
4,5,"African Leadership College, Mauritius",Flic en flac,Port Louis,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
5,6,"African Leadership College, Mauritius",Beau bassin,Port Louis,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
6,7,"African Leadership College, Mauritius","Village Moka, Mauritius",Mahogany Shopping Promenade,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
7,8,"African Leadership College, Mauritius","SSRN Hospital, Pamplemousses",,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
8,9,"African Leadership College, Mauritius",Beau Bassin Rose Hill,Port Louis,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
9,10,"African Leadership College, Mauritius",Quatre Bornes,Beau Bassin Rose Hill,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1


In [52]:
# Geocode all locations
if LOAD_PRE_PROCESSED_DATA:
    routes_df_full_geolocation = pd.read_csv('export/routes_df_full_geolocation.csv')
else:
    routes_df_full_geolocation = routes_df_full.apply(
        lambda row: parse_distance_matrix_response(
            row['Origin'], row['Destination'], row['Id']
            ), axis='columns', result_type='expand')
    routes_df_full_geolocation.to_csv('export/routes_df_full_geolocation.csv')

In [53]:
routes_df_full_geolocation

,id,distance,duration,full_direction_details,success_distance_matrix,match_found_distance_matrix
0,1,2591,273,"[{'bounds': {'northeast': {'lat': -20.0983642,...",True,1
1,2,15105,1141,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1
2,3,3653,431,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1
3,4,16828,1154,"[{'bounds': {'northeast': {'lat': -20.0089358,...",True,1
4,5,47666,3225,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1
5,6,22681,2021,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1
6,7,23293,1657,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1
7,8,2240,223,"[{'bounds': {'northeast': {'lat': -20.0950644,...",True,1
8,9,22681,2021,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1
9,10,32244,1921,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1


In [54]:
routes_df_full

,Id,Origin,Destination,Transits 1,Locations,longitude,latitude,formatted_address,full_details,success_geocode,match_found_geocode
0,1,"African Leadership College, Mauritius",Mahogany Shopping Promenade,,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
1,2,"African Leadership College, Mauritius",Port Louis,Mahogany Shopping Promenade,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
2,3,"African Leadership College, Mauritius",Pamplemousses,Mahogany Shopping Promenade,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
3,4,"African Leadership College, Mauritius",Grand baie,Mahogany Shopping Promenade,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
4,5,"African Leadership College, Mauritius",Flic en flac,Port Louis,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
5,6,"African Leadership College, Mauritius",Beau bassin,Port Louis,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
6,7,"African Leadership College, Mauritius","Village Moka, Mauritius",Mahogany Shopping Promenade,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
7,8,"African Leadership College, Mauritius","SSRN Hospital, Pamplemousses",,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
8,9,"African Leadership College, Mauritius",Beau Bassin Rose Hill,Port Louis,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
9,10,"African Leadership College, Mauritius",Quatre Bornes,Beau Bassin Rose Hill,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1


In [55]:
routes_df_full_geolocation = routes_df_full_geolocation.merge(routes_df_full, left_on='id', right_on='Id')

In [56]:
routes_df_full_geolocation

,id,distance,duration,full_direction_details,success_distance_matrix,match_found_distance_matrix,Id,Origin,Destination,Transits 1,Locations,longitude,latitude,formatted_address,full_details,success_geocode,match_found_geocode
0,1,2591,273,"[{'bounds': {'northeast': {'lat': -20.0983642,...",True,1,1,"African Leadership College, Mauritius",Mahogany Shopping Promenade,,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
1,2,15105,1141,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1,2,"African Leadership College, Mauritius",Port Louis,Mahogany Shopping Promenade,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
2,3,3653,431,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1,3,"African Leadership College, Mauritius",Pamplemousses,Mahogany Shopping Promenade,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
3,4,16828,1154,"[{'bounds': {'northeast': {'lat': -20.0089358,...",True,1,4,"African Leadership College, Mauritius",Grand baie,Mahogany Shopping Promenade,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
4,5,47666,3225,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1,5,"African Leadership College, Mauritius",Flic en flac,Port Louis,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
5,6,22681,2021,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1,6,"African Leadership College, Mauritius",Beau bassin,Port Louis,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
6,7,23293,1657,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1,7,"African Leadership College, Mauritius","Village Moka, Mauritius",Mahogany Shopping Promenade,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
7,8,2240,223,"[{'bounds': {'northeast': {'lat': -20.0950644,...",True,1,8,"African Leadership College, Mauritius","SSRN Hospital, Pamplemousses",,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
8,9,22681,2021,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1,9,"African Leadership College, Mauritius",Beau Bassin Rose Hill,Port Louis,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
9,10,32244,1921,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1,10,"African Leadership College, Mauritius",Quatre Bornes,Beau Bassin Rose Hill,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1


In [57]:
routes_df_full_geolocation.drop(['Id'], axis='columns', inplace=True)

In [59]:
routes_df_full_geolocation.to_csv('export/routes_df_full_geolocation_processed.csv')

In [67]:
routes_df_full_geolocation.columns

Index(['id', 'distance', 'duration', 'full_direction_details',
       'success_distance_matrix', 'match_found_distance_matrix', 'Origin',
       'Destination', 'Transits 1', 'Locations', 'longitude', 'latitude',
       'formatted_address', 'full_details', 'success_geocode',
       'match_found_geocode'],
      dtype='object')

In [62]:
routes_df_full_geolocation

,id,distance,duration,full_direction_details,success_distance_matrix,match_found_distance_matrix,Origin,Destination,Transits 1,Locations,longitude,latitude,formatted_address,full_details,success_geocode,match_found_geocode
0,1,2591,273,"[{'bounds': {'northeast': {'lat': -20.0983642,...",True,1,"African Leadership College, Mauritius",Mahogany Shopping Promenade,,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
1,2,15105,1141,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1,"African Leadership College, Mauritius",Port Louis,Mahogany Shopping Promenade,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
2,3,3653,431,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1,"African Leadership College, Mauritius",Pamplemousses,Mahogany Shopping Promenade,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
3,4,16828,1154,"[{'bounds': {'northeast': {'lat': -20.0089358,...",True,1,"African Leadership College, Mauritius",Grand baie,Mahogany Shopping Promenade,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
4,5,47666,3225,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1,"African Leadership College, Mauritius",Flic en flac,Port Louis,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
5,6,22681,2021,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1,"African Leadership College, Mauritius",Beau bassin,Port Louis,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
6,7,23293,1657,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1,"African Leadership College, Mauritius","Village Moka, Mauritius",Mahogany Shopping Promenade,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
7,8,2240,223,"[{'bounds': {'northeast': {'lat': -20.0950644,...",True,1,"African Leadership College, Mauritius","SSRN Hospital, Pamplemousses",,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
8,9,22681,2021,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1,"African Leadership College, Mauritius",Beau Bassin Rose Hill,Port Louis,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
9,10,32244,1921,"[{'bounds': {'northeast': {'lat': -20.0986227,...",True,1,"African Leadership College, Mauritius",Quatre Bornes,Beau Bassin Rose Hill,"African Leadership College, Mauritius",57.56245,-20.102035,"21001, Powder Mill Road, Pamplemousses, Mauritius",[{'address_components': [{'long_name': 'Pample...,True,1
